**Homework 4**

In [46]:
# Let's import first all the libraries needed (I updated this cell for every new question)

from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

from sklearn.ensemble import StackingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
import pandas as pd

In [47]:
# Question 1

# This homework is a continuation of HW 3. Use the same Auto.csv dataset as in HW3 and the binary variable mpg_high_low you created in HW 3

# Solution

# We start uploading the dataset. Basically I am coping the code from HM 3 for the binary variable mpg_high_low
from google.colab import files
uploaded = files.upload()

auto = pd.read_csv("Auto.csv")
auto.info()

Saving Auto.csv to Auto (4).csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392 entries, 0 to 391
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           392 non-null    float64
 1   cylinders     392 non-null    int64  
 2   displacement  392 non-null    float64
 3   horsepower    392 non-null    int64  
 4   weight        392 non-null    int64  
 5   acceleration  392 non-null    float64
 6   year          392 non-null    int64  
 7   origin        392 non-null    int64  
 8   name          392 non-null    object 
dtypes: float64(3), int64(5), object(1)
memory usage: 27.7+ KB


In [48]:
# Create the binary 'mpg_high_low' column
auto['mpg_high_low'] = ['High' if mpg > 30 else 'Low' for mpg in auto['mpg']]

# Convert the column to category type
auto['mpg_high_low'] = auto['mpg_high_low'].astype('category')

# Preview the column and its type
print(auto[['mpg', 'mpg_high_low']].head())
print(f"'mpg_high_low' type: {auto['mpg_high_low'].dtype}")

    mpg mpg_high_low
0  18.0          Low
1  15.0          Low
2  18.0          Low
3  16.0          Low
4  17.0          Low
'mpg_high_low' type: category


In [49]:
# Question 2

# Split the dataset into 75% training and 25% test and use 10 fold cross validation for the models below

# Solution

# Drop missing values. This is similar to HM 3, question 4, but without appying the 10 fold cross validation
auto = auto.dropna()

# Encode the binary target (Low=0, High=1)
le = LabelEncoder()
auto['mpg_high_low_encoded'] = le.fit_transform(auto['mpg_high_low'])

# Define X and y
X = auto.drop(columns=['mpg_high_low', 'mpg_high_low_encoded', 'mpg', 'name'])
y = auto['mpg_high_low_encoded']

# Stratified train-test split (75/25)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

In [50]:
# Question 3

# Fit an SVM model to the training set to predict mpg_high_low using all the other features/variables except mpg, year, origin, and name.
# Use a rbf kernel and cost parameter found by tuning using grid search of 10 evenly linearly spaced numbers in the range 0.1 to 100 and the
# gamma parameter found by searching 10 evenly logarithmically spaced numbers with a start value of -9 and stop value of 3 (hint: use numpy logspace).
# Predict the mpg_high_low using the test dataset and report the Accuracy, Precision, Recall, Specificity, and F1 measure.

# Solution

# Prepare GridSearch values
C_values = np.linspace(0.1, 100, 10)                         # Linear scale
gamma_values = np.logspace(-9, 3, 10, base=10)              # Logarithmic scale

# Create parameter grid
param_grid = {'C': C_values, 'gamma': gamma_values, 'kernel': ['rbf']}

# Initialize and fit GridSearch
svm = SVC()
grid_search = GridSearchCV(svm, param_grid, scoring='accuracy', cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)


GridSearchCV(cv=5, estimator=SVC(), n_jobs=-1,
             param_grid={'C': array([  0.1,  11.2,  22.3,  33.4,  44.5,  55.6,  66.7,  77.8,  88.9,
       100. ]),
                         'gamma': array([1.00000000e-09, 2.15443469e-08, 4.64158883e-07, 1.00000000e-05,
       2.15443469e-04, 4.64158883e-03, 1.00000000e-01, 2.15443469e+00,
       4.64158883e+01, 1.00000000e+03]),
                         'kernel': ['rbf']},
             scoring='accuracy')

In [51]:
# Predict using best model
best_svm = grid_search.best_estimator_
y_pred = best_svm.predict(X_test)

# Evaluate metrics
cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
specificity = tn / (tn + fp)
f1 = f1_score(y_test, y_pred)

# Print results
print("Best Parameters:", grid_search.best_params_)
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"F1 Score: {f1:.4f}")

Best Parameters: {'C': np.float64(22.300000000000004), 'gamma': np.float64(0.00021544346900318823), 'kernel': 'rbf'}
Accuracy: 0.8367
Precision: 0.9067
Recall: 0.8831
Specificity: 0.6667
F1 Score: 0.8947


In [52]:
# Question 4

# Fit a decision tree model to the training set to predict mpg_high_low using all the other features/variables except mpg, year, origin, and name.
# Predict the mpg_high_low using the test dataset and report the Accuracy, Precision, Recall, Specificity, and F1 measure.

# Solution

# Initialize the Decision Tree
dt_model = DecisionTreeClassifier(random_state=42)

# Train the model
dt_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_dt = dt_model.predict(X_test)

In [53]:
# Compute confusion matrix
cm_dt = confusion_matrix(y_test, y_pred_dt)
tn, fp, fn, tp = cm_dt.ravel()

# Calculate performance metrics
accuracy_dt = accuracy_score(y_test, y_pred_dt)
precision_dt = precision_score(y_test, y_pred_dt)
recall_dt = recall_score(y_test, y_pred_dt)
specificity_dt = tn / (tn + fp)
f1_dt = f1_score(y_test, y_pred_dt)

# Display results
print("Decision Tree Model Performance:")
print(f"Accuracy: {accuracy_dt:.4f}")
print(f"Precision: {precision_dt:.4f}")
print(f"Recall: {recall_dt:.4f}")
print(f"Specificity: {specificity_dt:.4f}")
print(f"F1 Score: {f1_dt:.4f}")

Decision Tree Model Performance:
Accuracy: 0.8878
Precision: 0.9342
Recall: 0.9221
Specificity: 0.7619
F1 Score: 0.9281


In [54]:
# Question 5

# Fit a Random Forest model to the training data to predict mpg_high_low using all the other features/variables except mpg, year, origin, and name.
# Use a n_estimator parameter found by searching amongst the values 50, 100, 200, 500 and max_depth parameter found by searching over the values 2,5,10 and 15.
# Predict the mpg_high_low using the test dataset.

# Solution

# Define parameter grid
param_grid_rf = {
    'n_estimators': [50, 100, 200, 500],
    'max_depth': [2, 5, 10, 15]
}

# Initialize model and GridSearch
rf = RandomForestClassifier(random_state=42)
grid_rf = GridSearchCV(rf, param_grid_rf, scoring='accuracy', cv=5, n_jobs=-1)
grid_rf.fit(X_train, y_train)

# Get best model and predict
best_rf = grid_rf.best_estimator_
y_pred_rf = best_rf.predict(X_test)

In [55]:
# Evaluate
cm_rf = confusion_matrix(y_test, y_pred_rf)
tn, fp, fn, tp = cm_rf.ravel()

accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf)
recall_rf = recall_score(y_test, y_pred_rf)
specificity_rf = tn / (tn + fp)
f1_rf = f1_score(y_test, y_pred_rf)

# Print results
print("Best Random Forest Parameters:", grid_rf.best_params_)
print("Random Forest Model Performance:")
print(f"Accuracy: {accuracy_rf:.4f}")
print(f"Precision: {precision_rf:.4f}")
print(f"Recall: {recall_rf:.4f}")
print(f"Specificity: {specificity_rf:.4f}")
print(f"F1 Score: {f1_rf:.4f}")

Best Random Forest Parameters: {'max_depth': 5, 'n_estimators': 100}
Random Forest Model Performance:
Accuracy: 0.8980
Precision: 0.9351
Recall: 0.9351
Specificity: 0.7619
F1 Score: 0.9351


In [56]:
# Question 6

# Fit a XGBoost model to the training data to predict mpg_high_low using all the other features/variables except mpg, year, origin, and name.
# Use a learning rate found by tuning using grid search of 10 evenly linearly spaced numbers in the range 0.1 to 1.
# Report the accuracy, precision, recall, specificity, F1 score and AUC.

# Solusion

# Define parameter grid for learning rate (eta)
learning_rates = np.linspace(0.1, 1.0, 10)
param_grid_xgb = {'learning_rate': learning_rates}

# Initialize model and grid search
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
grid_xgb = GridSearchCV(xgb_model, param_grid_xgb, scoring='accuracy', cv=5, n_jobs=-1)
grid_xgb.fit(X_train, y_train)

# Get best estimator and make predictions
best_xgb = grid_xgb.best_estimator_
y_pred_xgb = best_xgb.predict(X_test)
y_proba_xgb = best_xgb.predict_proba(X_test)[:, 1]  # For AUC

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [02:59:26] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [57]:
# Evaluate
cm_xgb = confusion_matrix(y_test, y_pred_xgb)
tn, fp, fn, tp = cm_xgb.ravel()

accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
precision_xgb = precision_score(y_test, y_pred_xgb)
recall_xgb = recall_score(y_test, y_pred_xgb)
specificity_xgb = tn / (tn + fp)
f1_xgb = f1_score(y_test, y_pred_xgb)
auc_xgb = roc_auc_score(y_test, y_proba_xgb)

# Print results
print("Best XGBoost Learning Rate:", grid_xgb.best_params_)
print("XGBoost Model Performance:")
print(f"Accuracy: {accuracy_xgb:.4f}")
print(f"Precision: {precision_xgb:.4f}")
print(f"Recall: {recall_xgb:.4f}")
print(f"Specificity: {specificity_xgb:.4f}")
print(f"F1 Score: {f1_xgb:.4f}")
print(f"AUC: {auc_xgb:.4f}")

Best XGBoost Learning Rate: {'learning_rate': np.float64(0.1)}
XGBoost Model Performance:
Accuracy: 0.9184
Precision: 0.9481
Recall: 0.9481
Specificity: 0.8095
F1 Score: 0.9481
AUC: 0.9524


In [58]:
# Question 7

# Fit a Stacked Classifier model to the training data to predict mpg_high_low using all the other features/variables except mpg, year, origin, and name.
# The models you need to stack are – SVM, decision tree, KNN, and Naïve Bayes. Report the accuracy, precision, recall, specificity and F1 score.

# Solution

# Define base estimators
estimators = [
    ('svm', SVC(kernel='rbf', probability=True, random_state=42)),
    ('dt', DecisionTreeClassifier(random_state=42)),
    ('knn', KNeighborsClassifier()),
    ('nb', GaussianNB())
]

# Define final estimator
final_estimator = LogisticRegression()

# Build stacked classifier
stacked_model = StackingClassifier(
    estimators=estimators,
    final_estimator=final_estimator,
    cv=5,
    n_jobs=-1
)

# Train the model
stacked_model.fit(X_train, y_train)

# Predict
y_pred_stack = stacked_model.predict(X_test)

In [59]:
# Evaluate
cm_stack = confusion_matrix(y_test, y_pred_stack)
tn, fp, fn, tp = cm_stack.ravel()

accuracy_stack = accuracy_score(y_test, y_pred_stack)
precision_stack = precision_score(y_test, y_pred_stack)
recall_stack = recall_score(y_test, y_pred_stack)
specificity_stack = tn / (tn + fp)
f1_stack = f1_score(y_test, y_pred_stack)

# Print results
print("Stacked Classifier Performance:")
print(f"Accuracy: {accuracy_stack:.4f}")
print(f"Precision: {precision_stack:.4f}")
print(f"Recall: {recall_stack:.4f}")
print(f"Specificity: {specificity_stack:.4f}")
print(f"F1 Score: {f1_stack:.4f}")

Stacked Classifier Performance:
Accuracy: 0.9082
Precision: 0.9359
Recall: 0.9481
Specificity: 0.7619
F1 Score: 0.9419


In [61]:
# Question 8

# Summarize the performance of the all the above models by creating a dataframe with 6 columns – Model_Name, Accuracy, Precision, Recall,
# Specificity, F1 Score. The data frame should contain one row for each model you built above with each of the columns filled in with the
# appropriate metric. Print out the dataframe. Which model performed the best from an accuracy point of view and which model performed best
# from a recall point of view? Of all the models you built in HW3 and HW4 which one performed best from an F1 score perspective?

# Solution

# Final model metrics (I copied the results from Homewrok 3 and also the above results + doing the analysis)
model_summary = [
    ['Logistic Regression', 0.877551, 0.901235, 0.948052, 0.380952, 0.924051],
    ['Naive Bayes', 0.857143, 0.943662, 0.870130, 0.904762, 0.905405],
    ['KNN', 0.877551, 0.911392, 0.935065, 0.666667, 0.923077],
    ['LDA', 0.846939, 0.852273, 0.974026, 0.380952, 0.909091],
    ['SVM (RBF)', 0.8367, 0.9067, 0.8331, 0.6667, 0.8947],
    ['Decision Tree', 0.8878, 0.9342, 0.9221, 0.7619, 0.9281],
    ['Random Forest', 0.8980, 0.9351, 0.9351, 0.7619, 0.9351],
    ['XGBoost', 0.9184, 0.9481, 0.9481, 0.8095, 0.9481],
    ['Stacked Classifier', 0.9082, 0.9359, 0.9481, 0.7619, 0.9419]
]

# Create dataframe
summary_df = pd.DataFrame(model_summary, columns=[
    'Model_Name', 'Accuracy', 'Precision', 'Recall', 'Specificity', 'F1 Score'
])

In [62]:
# Display summary
print("Model Performance Summary:")
print(summary_df)

# Best models
best_accuracy_model = summary_df.loc[summary_df['Accuracy'].idxmax(), 'Model_Name']
best_recall_model = summary_df.loc[summary_df['Recall'].idxmax(), 'Model_Name']
best_f1_model = summary_df.loc[summary_df['F1 Score'].idxmax(), 'Model_Name']

# Final answers
print("\n Best model by Accuracy:", best_accuracy_model)
print(" Best model by Recall:", best_recall_model)
print(" Best model by F1 Score (HW3 + HW4):", best_f1_model)

Model Performance Summary:
            Model_Name  Accuracy  Precision    Recall  Specificity  F1 Score
0  Logistic Regression  0.877551   0.901235  0.948052     0.380952  0.924051
1          Naive Bayes  0.857143   0.943662  0.870130     0.904762  0.905405
2                  KNN  0.877551   0.911392  0.935065     0.666667  0.923077
3                  LDA  0.846939   0.852273  0.974026     0.380952  0.909091
4            SVM (RBF)  0.836700   0.906700  0.833100     0.666700  0.894700
5        Decision Tree  0.887800   0.934200  0.922100     0.761900  0.928100
6        Random Forest  0.898000   0.935100  0.935100     0.761900  0.935100
7              XGBoost  0.918400   0.948100  0.948100     0.809500  0.948100
8   Stacked Classifier  0.908200   0.935900  0.948100     0.761900  0.941900

 Best model by Accuracy: XGBoost
 Best model by Recall: LDA
 Best model by F1 Score (HW3 + HW4): XGBoost


In [63]:
# I created above a summary table that shows the performance of all models built in HW3 and HW4. The table includes Accuracy, Precision, Recall,
# Specificity, and F1 Score for each model. After comparing them we can say that XGBoost was the best model in terms of Accuracy and F1 Score.
# While LDA had the highest Recall, meaning it best identified the "High" mpg class.
# This helped identify which models were strongest for different evaluation goals.